In [7]:
import numpy as np 
import pandas as pd 
import os
import ast
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict, GridSearchCV, RandomizedSearchCV

## Preprocessing 
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.pipeline import Pipeline, FeatureUnion
!pip install sklearn_features
from sklearn_features.transformers import DataFrameSelector
from sklearn.linear_model import LinearRegression, SGDRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score , mean_squared_error, mean_absolute_error

## XGBoost
import xgboost as xgb

In [8]:
!pip install sklearn_features

In [9]:
df_housing= pd.read_csv(r'D:\projects\ml projects\data\realEstate.csv\realEstate.csv')

In [10]:
df_housing.shape


(6762, 30)

In [11]:
dict_data = df_housing['location'].to_dict()

lat = []
lang = []
for k, v in dict_data.items():
    e = eval(v)
    lat.append(e['lat'])
    lang.append(e['lng'])
    
    
df_housing['lat'] = lat
df_housing['long'] = lang


In [12]:
df_housing.head()

,user_id,id,title,price,content,imgs,refresh,beds,livings,wc,...,advertiser_type,create_time,review,profileImg,UserName,iam_verified,onMarket,IsRent,lat,long
0,1495817,4596035,شقة للإيجار في شارع التغلبي ، حي النظيم ، الري...,20000,1666 كل شهر,"['014958174_1659666836308.jpg', '014958178_165...","['2022-08-06', '12:22:15']",3,0.0,2,...,owner,"['2022-08-05', '02:44:23']",5.00,NaN,Thamer,True,17,False,24.800930,46.896890
1,496663,4599813,شقة للإيجار في شارع الاقتصاد ، حي الفيحاء ، ال...,25000,شقة ارضية للإيجار بحي الفيحاء شارع الاقتصاد \r...,"['004966630_1659781493595.jpg', '004966636_165...","['2022-08-06', '10:27:57']",3,1.0,2,...,exclusive_marketer,"['2022-08-06', '10:27:56']",4.33,yes,الاركان الثابته للعقارات,True,5,True,24.687521,46.807558
2,2412431,4554519,شقة للإيجار في شارع أبي الحسن الأسفرائيني ، حي...,22000,شقه للايجار حي الرمال \r\nمكون\r\nالدور ثالث\r...,"['023243548_1658701693777.jpg', '023243549_165...","['2022-08-06', '06:59:03']",3,1.0,2,...,normal_marketer,"['2022-07-24', '22:27:27']",4.67,NaN,مكتب منزلك الفاخر,True,15,True,24.921463,46.806270
3,1906500,4120004,شقة للإيجار في شارع أسيد بن ثعلبة ، حي العقيق ...,38000,شقه عوائل نظيفه جدا \r\nغرفه كبيره وصاله كبيره...,"['019065004_1659758037501.jpg', '019065000_165...","['2022-08-06', '03:52:58']",1,1.0,1,...,owner,"['2022-03-10', '09:17:01']",4.17,NaN,ابو مصطفى,True,165,False,24.780059,46.630602
4,1741562,4498954,شقة للإيجار في شارع العباس بن عبدالمطلب ، حي ا...,25000,غرفه وصاله وحمام راكب مطبخ\r\nراكب مكيفات مؤثث...,"['017415629_1657025837185.jpg', '017415620_165...","['2022-08-03', '10:53:47']",1,1.0,1,...,normal_marketer,"['2022-07-05', '13:03:54']",4.42,NaN,عقار المستقبل,True,48,False,24.771793,46.698757


In [13]:
df_housing.tail()

,user_id,id,title,price,content,imgs,refresh,beds,livings,wc,...,advertiser_type,create_time,review,profileImg,UserName,iam_verified,onMarket,IsRent,lat,long
6757,1958104,4538503,شقة للإيجار في شارع مطلق بن عجيبان ، حي النرجس...,60000,شقق فاخره للايجار بحي النرجس بفيله سكنيه \r\n•...,NaN,"['2022-08-05', '10:34:28']",3,1.0,2,...,exclusive_marketer,"['2022-07-20', '12:31:13']",4.31,yes,مكتب عقاري,True,33,False,24.866976,46.649873
6758,395654,3130523,شقة للإيجار في شارع الحويرة ، حي غبيرة ، الريا...,9800,للايجار شقة عزاب نظيفة في غبيراء ٩٨٠٠ريال سنوي...,NaN,"['2022-07-30', '10:22:11']",2,1.0,1,...,exclusive_marketer,"['2021-04-16', '00:55:25']",4.82,yes,بصمة نجدالعقارية,True,493,False,24.625145,46.735970
6759,292698,4453217,شقة للإيجار في شارع جليل ، حي اليرموك ، الرياض...,50000,شقة مؤثثة بالكامل دور اول مجلس وصالة وغرفتين ن...,NaN,"['2022-08-02', '15:42:17']",3,1.0,2,...,owner,"['2022-06-21', '23:35:16']",5.00,NaN,المشاريع الناجحة,True,62,False,24.807284,46.767262
6760,120924,4586116,شقة للإيجار في شارع محمد بن رزين ، حي النرجس ،...,40000,عرض رقم 1320\r\n\r\nبسم الله الرحمن الرحيم\r\n...,NaN,"['2022-08-05', '06:00:43']",2,2.0,2,...,exclusive_marketer,"['2022-08-02', '14:48:55']",4.54,yes,مؤسسة عثمان العبرة العقارية,True,20,False,24.870188,46.650692
6761,2110604,4025771,شقة للإيجار في شارع السلامه ، حي النرجس ، الري...,17000,غرفة وحمام \r\nمكيفات اسبيلت \r\nبحي النرجس,NaN,"['2022-08-03', '23:50:08']",1,0.0,1,...,owner,"['2022-02-11', '16:47:18']",4.54,NaN,Badr,True,192,False,24.843407,46.679985


In [14]:
price_corr = df_housing.corr()['price'].sort_values(ascending = False)
price_corr

price           1.000000
wc              0.558159
beds            0.345703
livings         0.318963
ac              0.234097
lat             0.201601
furnished       0.196255
ketchen         0.120305
street_width    0.115382
width           0.045184
length          0.019584
area            0.015561
onMarket        0.003216
id              0.001992
review         -0.009188
user_id        -0.020560
IsRent         -0.075994
age            -0.240437
long           -0.268415
Name: price, dtype: float64

In [15]:
df = df_housing.drop(['user_id', 'id', 'imgs', 'content', 'title', 'refresh', 'create_time', 'profileImg', 'UserName', 'iam_verified', 'path', 'last_update',  'district','location'], axis=1)
df

,price,beds,livings,wc,area,street_width,age,ketchen,ac,furnished,width,length,advertiser_type,review,onMarket,IsRent,lat,long
0,20000,3,0.0,2,225.0,15.0,9.0,1.0,0.0,0.0,15.0,15.0,owner,5.00,17,False,24.800930,46.896890
1,25000,3,1.0,2,130.0,30.0,12.0,1.0,0.0,0.0,NaN,NaN,exclusive_marketer,4.33,5,True,24.687521,46.807558
2,22000,3,1.0,2,200.0,25.0,NaN,0.0,0.0,0.0,10.0,20.0,normal_marketer,4.67,15,True,24.921463,46.806270
3,38000,1,1.0,1,120.0,34.0,0.0,1.0,1.0,0.0,NaN,NaN,owner,4.17,165,False,24.780059,46.630602
4,25000,1,1.0,1,60.0,39.0,9.0,1.0,1.0,0.0,7.0,9.0,normal_marketer,4.42,48,False,24.771793,46.698757
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6757,60000,3,1.0,2,180.0,18.0,0.0,1.0,1.0,0.0,NaN,NaN,exclusive_marketer,4.31,33,False,24.866976,46.649873
6758,9800,2,1.0,1,90.0,5.0,25.0,0.0,0.0,0.0,NaN,NaN,exclusive_marketer,4.82,493,False,24.625145,46.735970
6759,50000,3,1.0,2,120.0,20.0,5.0,1.0,1.0,1.0,11.0,11.0,owner,5.00,62,False,24.807284,46.767262
6760,40000,2,2.0,2,200.0,15.0,1.0,1.0,1.0,0.0,20.0,10.0,exclusive_marketer,4.54,20,False,24.870188,46.650692


In [16]:
df.shape

(6762, 18)

In [17]:
df.describe().round(2)

,price,beds,livings,wc,area,street_width,age,ketchen,ac,furnished,width,length,review,onMarket,lat,long
count,6762.00,6762.00,6760.00,6762.00,6587.00,6552.00,6057.00,6745.00,6743.00,6717.00,4662.00,4662.00,6363.00,6762.00,6762.00,6762.00
mean,43274.64,2.57,0.99,2.07,1880.13,23.14,2.86,0.87,0.72,0.08,17.85,47.17,4.34,49.80,24.79,46.69
std,28860.91,1.12,0.46,0.93,136920.18,13.79,4.66,0.34,0.45,0.27,35.43,1627.62,0.48,71.21,0.08,0.08
min,55.00,1.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,1.00,1.00,1.00,24.40,46.32
25%,25000.00,2.00,1.00,1.00,98.00,15.00,0.00,1.00,0.00,0.00,8.00,9.00,4.20,19.00,24.77,46.62
50%,35000.00,3.00,1.00,2.00,145.00,20.00,1.00,1.00,1.00,0.00,11.00,12.00,4.38,32.00,24.81,46.67
75%,55000.00,3.00,1.00,3.00,180.00,30.00,4.00,1.00,1.00,0.00,15.00,18.00,4.56,61.00,24.83,46.76
max,450000.00,7.00,5.00,5.00,11111100.00,100.00,36.00,1.00,1.00,1.00,1000.00,111111.00,5.00,2279.00,25.18,46.91


In [18]:
df.describe().shape

(8, 16)

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6762 entries, 0 to 6761
Data columns (total 18 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   price            6762 non-null   int64  
 1   beds             6762 non-null   int64  
 2   livings          6760 non-null   float64
 3   wc               6762 non-null   int64  
 4   area             6587 non-null   float64
 5   street_width     6552 non-null   float64
 6   age              6057 non-null   float64
 7   ketchen          6745 non-null   float64
 8   ac               6743 non-null   float64
 9   furnished        6717 non-null   float64
 10  width            4662 non-null   float64
 11  length           4662 non-null   float64
 12  advertiser_type  6522 non-null   object 
 13  review           6363 non-null   float64
 14  onMarket         6762 non-null   int64  
 15  IsRent           6762 non-null   bool   
 16  lat              6762 non-null   float64
 17  long          

In [20]:
df.isna().sum()

price                 0
beds                  0
livings               2
wc                    0
area                175
street_width        210
age                 705
ketchen              17
ac                   19
furnished            45
width              2100
length             2100
advertiser_type     240
review              399
onMarket              0
IsRent                0
lat                   0
long                  0
dtype: int64

In [21]:
df.dtypes.value_counts()

float64    12
int64       4
object      1
bool        1
dtype: int64

In [22]:
x = df.drop(['price'], axis=1)
y = df['price']

In [23]:
price_corr = df.corr()['price'].sort_values(ascending = False)
price_corr

price           1.000000
wc              0.558159
beds            0.345703
livings         0.318963
ac              0.234097
lat             0.201601
furnished       0.196255
ketchen         0.120305
street_width    0.115382
width           0.045184
length          0.019584
area            0.015561
onMarket        0.003216
review         -0.009188
IsRent         -0.075994
age            -0.240437
long           -0.268415
Name: price, dtype: float64

In [24]:
### Feature Engineering 
'I will try to add some features more corelated and more useful to the target'

df['beds per livings'] = df['beds']/df['livings']
df['wc  per livings'] = df['wc']/df['livings']
df["total_area"] = df["width"] * df["length"]

In [25]:
price_corr = df.corr()['price'].sort_values(ascending = False)
price_corr

price               1.000000
wc                  0.558159
wc  per livings     0.427698
beds                0.345703
livings             0.318963
ac                  0.234097
lat                 0.201601
furnished           0.196255
beds per livings    0.181471
ketchen             0.120305
street_width        0.115382
width               0.045184
length              0.019584
total_area          0.017767
area                0.015561
onMarket            0.003216
review             -0.009188
IsRent             -0.075994
age                -0.240437
long               -0.268415
Name: price, dtype: float64

In [43]:

X_train_full, X_test, y_train_full, y_test = train_test_split(x, y, test_size=0.2,random_state=42)
X_train, X_valid, y_train,  y_valid = train_test_split(X_train_full, y_train_full, test_size = 0.2,random_state=42)

In [44]:
## Separete the columns according to type (numerical or categorical)
num_cols = [col for col in  X_train.columns 
             if X_train[col].dtype in ['float64', 'int64']]

print('Numerical Columns : \n', num_cols)
print('**'*30)

categ_cols = [col for col in  X_train.columns 
             if X_train[col].dtype not in ['float64', 'int64']]

print('categorical Columns : \n', categ_cols)
print('**'*30)


Numerical Columns : 
 ['beds', 'livings', 'wc', 'area', 'street_width', 'age', 'ketchen', 'ac', 'furnished', 'width', 'length', 'review', 'onMarket', 'lat', 'long']
************************************************************
categorical Columns : 
 ['advertiser_type', 'IsRent']
************************************************************


In [45]:
'''pyt
ohe = OneHotEncoder(sparse=False)
df_ohe = ohe.fit_transform(df[['advertiser_type']])


SyntaxError: EOF while scanning triple-quoted string literal (546190296.py, line 3)

In [46]:
## We can get much much easier like the following
## numerical pipeline
num_pipeline = Pipeline([
                    ('selector', DataFrameSelector(num_cols)),    ## select only these columns
                    ('imputer', SimpleImputer(strategy='median')),
                    ('scaler', StandardScaler())])

## categorical pipeline
categ_pipeline = Pipeline(steps=[
            ('selector', DataFrameSelector(categ_cols)),    ## select only these columns
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('OHE', OneHotEncoder(sparse=False))])

## concatenate both two pipelines
total_pipeline = FeatureUnion(transformer_list=[
                                ('num_pip', num_pipeline),
                                ('categ_pipeline', categ_pipeline)])

## deal with (total_pipeline) as an instance -- fit and transform to train dataset and transform only to other datasets
X_train_final = total_pipeline.fit_transform(X_train_full)
X_test_final = total_pipeline.transform(X_test)                 ### Every thing is processed :D

In [47]:
X_train_final.shape ,X_valid_final.shape, X_test_final.shape 

((5409, 21), (1082, 21), (1353, 21))

###**Building ML Models**


##**Linear Regression**




In [54]:
lin_reg = LinearRegression()
lin_reg.fit(X_train_final, y_train_full)

LinearRegression()

In [55]:
y_predict_train = lin_reg.predict(X_train_final)
y_predict_valid = lin_reg.predict(X_valid_final)

In [85]:
r2_train_lin = r2_score(y_train_full, y_predict_train)
r2_valid_lin = r2_score(y_valid, y_predict_valid)
print("r2_train_lin >.... ", r2_train_lin)
print("r2_valid_lin >.... ", r2_valid_lin)

r2_train_lin >....  0.45763084715285407
r2_valid_lin >....  0.4775712974889611


In [87]:
rmse_train_lin = (mean_squared_error(y_train_full, y_predict_train))**0.5
rmse_valid_lin = (mean_squared_error(y_valid, y_predict_valid))**0.5
print("rmse_train_lin >.....", rmse_train_lin)
print("rmse_valid_lin >.....", rmse_valid_lin)

rmse_train_lin >..... 21513.800724645986
rmse_valid_lin >..... 21262.95611967386


## ***SGD Regressor***


In [88]:
sgd_reg = SGDRegressor(penalty=None, eta0=0.001, shuffle=True, max_iter=1000)  
sgd_reg.fit(X_train_final, y_train_full)


SGDRegressor(eta0=0.001, penalty=None)

In [89]:
y_pred_train_sgd = sgd_reg.predict(X_train_final) 
y_pred_valid_sgd = sgd_reg.predict(X_valid_final) 

In [90]:
r2_train_sgd = r2_score(y_train_full, y_pred_train_sgd)
r2_valid_sgd = r2_score(y_valid, y_pred_valid_sgd)
print("r2_train_sgd >....", r2_train_sgd)
print("r2_valid_sgd >....", r2_valid_sgd)

r2_train_sgd >.... 0.4576171794596374
r2_valid_sgd >.... 0.477496147167925


##**Random Forest Regressor**

In [91]:

rf_reg = RandomForestRegressor( n_estimators=100, max_depth=20)
rf_reg.fit(X_train_final, y_train_full)
y_pred_train_rf = rf_reg.predict(X_train_final) 
y_pred_valid_rf = rf_reg.predict(X_valid_final) 

In [92]:
r2_score_train_rf = r2_score(y_train_full, y_pred_train_rf)

r2_score_valid_rf = r2_score(y_valid, y_pred_valid_rf)

print("r2_train_rf >....", r2_score_train_rf)
print("r2_valid_rf >....", r2_score_valid_rf)

r2_train_rf >.... 0.939488998483344
r2_valid_rf >.... 0.7729534896999805


In [93]:
rmse_train_rf = (mean_squared_error(y_train_full, y_pred_train_rf))**0.5
rmse_train_rf
rmse_valid_rf = (mean_squared_error(y_valid, y_pred_valid_rf))**0.5
rmse_valid_rf

print("rmse_train_rf=",rmse_train_rf)
print("rmse_valid_rf=",rmse_valid_rf)

rmse_train_rf= 7185.993524386583
rmse_valid_rf= 14017.402496747764


In [94]:
# Define the parameter grid
param_grid = {
    'n_estimators': [50, 100, 200, 300, 400],
    'max_depth': [5, 10, 15, 20, 25, 30],
    'min_samples_split': [2, 4, 6, 8],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2'],
    'bootstrap': [True, False]
}

In [95]:
# Create a RandomizedSearchCV object
grid_search = RandomizedSearchCV(estimator=rf_reg, param_distributions=param_grid, cv=5, n_iter=10, n_jobs=-1)
# Fit the object to the training data
grid_search.fit(X_train_final, y_train_full)

RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(max_depth=20),
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [5, 10, 15, 20, 25, 30],
                                        'max_features': ['auto', 'sqrt',
                                                         'log2'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 4, 6, 8],
                                        'n_estimators': [50, 100, 200, 300,
                                                         400]})

In [96]:
# Get the best parameters
best_params = grid_search.best_params_
print(best_params)

{'n_estimators': 400, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'log2', 'max_depth': 30, 'bootstrap': True}


In [97]:
# Create a new Random Forest regressor with the best parameters
rf_reg = RandomForestRegressor(**best_params)

In [98]:
# Fit the model to the training data
rf_reg.fit(X_train_final, y_train_full)
# Make predictions on the training and validation sets
y_pred_train_rf = rf_reg.predict(X_train_final)
y_pred_valid_rf = rf_reg.predict(X_valid_final)

In [99]:
# Calculate the R2 scores and RMSE for the training and validation sets
r2_score_train_rf = r2_score(y_train_full, y_pred_train_rf)
r2_score_valid_rf = r2_score(y_valid, y_pred_valid_rf)

rmse_train_rf = (mean_squared_error(y_train_full, y_pred_train_rf))**0.5
rmse_valid_rf = (mean_squared_error(y_valid, y_pred_valid_rf))**0.5

print("R2 score for training set: ", r2_score_train_rf)
print("R2 score for valid set: ", r2_score_valid_rf)
print("RMSE for training set: ", rmse_train_rf)
print("RMSE for valid set: ", rmse_valid_rf)

R2 score for training set:  0.9510696600836815
R2 score for valid set:  0.8868852655412317
RMSE for training set:  6461.878195046853
RMSE for valid set:  9893.950163199343


In [113]:
xgb_reg = xgb.XGBRegressor(objective='reg:squarederror', ## The most one according to your task (here, Multiclass class.)
                            learning_rate=0.03,           ## eta or learning_rate 
                            n_estimators=300,             ## number of trees
                            use_label_encoder=True)     ## only to remove the warning
xgb_reg.fit(X_train_final, y_train_full)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.03, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=300, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [117]:
## Tuning using GridSearchCV
params_xgb = {'n_estimators':np.arange(100, 1000, 100), 'learning_rate':np.arange(0.01, 0.1, 0.01)}
grid_xgb = GridSearchCV(estimator=xgb_reg, 
                        param_grid=params_xgb, 
                        cv=5, 
                        n_jobs=-1, 
                        scoring='neg_mean_squared_error', 
                        verbose=6)

grid_xgb.fit(X_train_final, y_train_full)   ## train

Fitting 5 folds for each of 81 candidates, totalling 405 fits


GridSearchCV(cv=5,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    feature_types=None, gamma=None, gpu_id=None,
                                    grow_policy=None, importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=0.03, m...
                                    max_depth=None, max_leaves=None,
                                    min_child_weight=None, missing=nan,
                                    monotone_constraints=None, n_estimators=300,
                                    n_jobs=None, num_parallel_tree=Non

In [118]:
best_parms_xgb = grid_xgb.best_params_
print('Best Parameter using XGB -- ', best_parms_xgb)

Best Parameter using XGB --  {'learning_rate': 0.02, 'n_estimators': 700}


In [119]:
xgb_best = grid_xgb.best_estimator_
print('Best Estimaror using XGB -- ', xgb_best)

Best Estimaror using XGB --  XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.02, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=700, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)


In [120]:
# Make predictions on the training and validation sets
y_pred_train_xgb = xgb_reg.predict(X_train_final)
y_pred_valid_xgb = xgb_reg.predict(X_valid_final)

In [121]:
# Calculate the R2 scores and RMSE for the training and validation sets
r2_score_train_xgb = r2_score(y_train_full, y_pred_train_xgb)
r2_score_valid_xgb = r2_score(y_valid, y_pred_valid_xgb)

rmse_train_xgb= (mean_squared_error(y_train_full, y_pred_train_xgb))**0.5
rmse_valid_xgb = (mean_squared_error(y_valid, y_pred_valid_xgb))**0.5

print("R2 score for training set: ", r2_score_train_xgb)
print("R2 score for valid set: ", r2_score_valid_xgb)
print("RMSE for training set: ", rmse_train_xgb)
print("RMSE for valid set: ", rmse_valid_xgb)

R2 score for training set:  0.8957017840413051
R2 score for valid set:  0.7493700848096438
RMSE for training set:  9434.265432032047
RMSE for valid set:  14727.416932908565
